# Function sandbox

This notebook has test for code and functions that have been moved to a separate module.

### Batch upsert
<div class="alert alert-danger">
Use functions from module instead of this version
</div>
Define a function to batch process insert or update queries:

Just a test with random data, use `execute=False` to print the query:

### Functions to read records in a spreadsheet

<div class="alert alert-danger">
Use functions from module instead of this version
</div>

We need a wrapping function to apply a lower level function (`create_record_function`) to all rows in a `worksheet` of the selected `workbook` using a dictionary `col_dictionary`, we add a `**kwargs` to pass additional arguments to the lower level function:


#### Insert into field_site table

This function will transform create an insert records from one row of the spreadsheet (`item`) using a column dictionary (`sw`). 

We need to consider:
- geom might be single or multiple points
- projection (SRID) is UTM GDA zone 55 or 56, latlong WGS84, or a different format
- elevation in m, or NULL 
- GPS uncertainty in meters, or NULL
- text description of GPS location, or NULL

Test this function with one workbook:

In [14]:
worksheet='Site'
filename='UNSWFireVegResponse_UplandBasalt_AlexThomsen+DK.xlsx'
col_definitions={'site_label':0, 'location_description':10,'utm_zone':11, 'xs':(12,), 'ys':(13,),
                 'gps_uncertainty_m':14,
                 'gps_geom_description':17,
                 'elevation':38, 'visit_date':(2,4,5)}
survey="UplandBasalt"

records = import_records_from_workbook(inputdir,filename,worksheet,col_definitions,create_field_site_record) 

len(records)

28

Check details from one record:

In [15]:
records[12]

{'site_label': 'MWL03',
 'location_description': 'Wynnes Rock Lookout Road',
 'gps_geom_description': '30 m transect for woody plants >2m tall, with two 5x5m subplots at either end (subplots 1 & 2) with 20x5m subplot in middle (subplot 3); non-woody spp and woodplants <2m tall counted in the two 5x5m subplots',
 'geom': "ST_GeomFromText('POINT(256134 6288811)', 28356)"}

#### Insert into field_visits table

This function will create an insert record from one row of the spreadsheet (`item`) using a column dictionary (`sw`). 

We need to consider:
- iterate over multiple visit dates in different columns
- add survey name to the record
- text description of visit, or NULL
- observerlist to be split into multiple names (list or array)

In [16]:
def create_field_visit_record(item,sw):
    site_label = item[sw['site_label']].value
    records = list()
    for k in sw['visit_date']:
        visit_date = item[k].value
        if site_label is not None and site_label != "Site":
            if isinstance(visit_date, datetime):
                record = {'visit_id': site_label, 'visit_date': visit_date}
                if 'survey' in sw.keys():
                    record['survey_name'] = sw['survey']
                for column in ('visit_description', 'mainobserver', 'observerlist','replicate_nr'):
                    if column in sw.keys():
                        val=item[sw[column]].value
                        if val is not None and val not in ('na','NA'):
                            if column=='observerlist':
                                val=val.split(',')
                            record[column] =  val
                records.append(record)
    return records

Test of the function with one workbook/worksheet:

In [17]:
worksheet='Site'
filename='UNSWFireVegResponse_UplandBasalt_AlexThomsen+DK.xlsx'
col_definitions={'site_label':0, 'location_description':10,'utm_zone':11, 'xs':(12,), 'ys':(13,),
                 'gps_uncertainty_m':14,
                 'gps_geom_description':17,
                 'observerlist':3,
                 'elevation':38, 'visit_date':(2,4,5,6,7,8,9),
                 'survey':"UplandBasalt"}

records = import_records_from_workbook(inputdir,filename,worksheet,col_definitions,create_field_visit_record) 

len(records)

42

In [18]:
records[21]

{'visit_id': 'MWL04',
 'visit_date': datetime.datetime(2000, 12, 14, 0, 0),
 'survey_name': 'UplandBasalt',
 'observerlist': ['Alexandria Thomsen', ' Stephan Wilson']}

### Import records to database
I create another function that will call the above functions to process data from a workbook into records that are then imported into the database.

This functions was renamed to `import_site_and_visit_records`.

This function passes the keyword arguments `**kwargs` to the next functions. This works, because the structure of both the `create_record_function`s is similar and we can define the column correspondence in the same dictionary as we will see in the examples below:
